In [ ]:
from io import BytesIO
import torch
from PIL import Image
import ipywidgets as widgets
import IPython.display as display
from ipywidgets import HTML
import base64

In [ ]:
model = torch.hub.load("bryandlee/animegan2-pytorch:main", "generator", device="cpu",pretrained="face_paint_512_v2").eval()
face2paint = torch.hub.load("bryandlee/animegan2-pytorch:main", "face2paint", device="cpu")

In [ ]:
button = widgets.Button(description="Let the magin begin ;)")
btn_upload = widgets.FileUpload(description='Upload Image',accept='image/*', multiple=False)
out_pl = widgets.Output()
image_format = "jpeg"
button.disabled = False

In [ ]:
def on_data_change(change):
    
    button.disabled = True
    with out_pl:
        display.clear_output()
        
    for fname in btn_upload.value:
        bytes_in = btn_upload.value[fname]['content']

        im_in = Image.open(BytesIO(bytes_in)).convert("RGB")
        im_out = face2paint(model, im_in, side_by_side=False)

        buffer_out = BytesIO()
        im_out.save(buffer_out, format=image_format)

        bytes_out = buffer_out.getvalue()
        wi1 = widgets.Image(value=bytes_in, format=image_format)
        wi2 = widgets.Image(value=bytes_out, format=image_format)

        wi1.layout.max_width = '500px'
        wi1.layout.max_height = '500px'
        wi2.layout.max_width = '500px'
        wi2.layout.max_height = '500px'
        sidebyside = widgets.HBox([wi1, wi2])
        with out_pl:
                display.display(sidebyside)
                
        #FILE
        filename = 'Anime.jpeg'
        b64 = base64.b64encode(bytes_out)
        payload = b64.decode()

        #BUTTONS
        html_buttons = '''<html>
        <head>
        <meta name="viewport" content="width=device-width, initial-scale=1">
        </head>
        <body>
        <a download="{filename}" href="data:text/csv;base64,{payload}" download>
        <button class="p-Widget jupyter-widgets jupyter-button widget-button mod-warning">Download File</button>
        </a>
        </body>
        </html>
        '''

        html_button = html_buttons.format(payload=payload,filename=filename)
        download_button=HTML(html_button)
        display.display(download_button)       
        
    btn_upload.value.clear()
    btn_upload._counter = 0
    button.disabled = False

In [ ]:
def reset_button(a):
    btn_upload.value.clear()
    btn_upload._counter = 0

In [ ]:
button.on_click(on_data_change)
button.on_click(reset_button)
display.display(btn_upload, button,out_pl)